## SIT789 Task-4.1 P

In [1]:
import numpy as np
import cv2 as cv
from sklearn.cluster import KMeans

class Dictionary(object):
    def __init__(self, name, img_filenames, num_words):
        self.name = name #name of your dictionary
        self.img_filenames = img_filenames #list of image filenames
        self.num_words = num_words #the number of words
        self.training_data = [] #this is the training data required by the K-Means algorithm
        self.words = [] #list of words, which are the centroids of clusters
    
    def learn(self):
        sift = cv.xfeatures2d.SIFT_create()
        num_keypoints = [] #this is used to store the number of keypoints in each image
        #load training images and compute SIFT descriptors
        for filename in self.img_filenames:
            img = cv.imread(filename)
            img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            list_des = sift.detectAndCompute(img_gray, None)[1]
            if list_des is None:
                num_keypoints.append(0)
            else:
                num_keypoints.append(len(list_des))
                for des in list_des:
                    self.training_data.append(des)

        #cluster SIFT descriptors using K-means algorithm
        kmeans = KMeans(self.num_words)
        kmeans.fit(self.training_data)
        self.words = kmeans.cluster_centers_
        #create word histograms for training images
        training_word_histograms = [] #list of word histograms of all training images
        index = 0
        for i in range(0, len(self.img_filenames)):
            #for each file, create a histogram
            histogram = np.zeros(self.num_words, np.float32)
            #if some keypoints exist
            if num_keypoints[i] > 0:
                for j in range(0, num_keypoints[i]):
                    histogram[kmeans.labels_[j + index]] += 1
                index += num_keypoints[i]
                histogram /= num_keypoints[i]
                training_word_histograms.append(histogram)
                    
        return training_word_histograms
    
    def create_word_histograms(self, img_filenames):
        sift = cv.xfeatures2d.SIFT_create()
        histograms = []
        for filename in img_filenames:
            img = cv.imread(filename)
            img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            descriptors = sift.detectAndCompute(img_gray, None)[1]

            histogram = np.zeros(self.num_words, np.float32) #word histogram for the input image
            if descriptors is not None:
                for des in descriptors:
                    #find the best matching word
                    min_distance = 1111111 #this can be any large number
                    matching_word_ID = -1 #initial matching_word_ID=-1 means no matching
                    for i in range(0, self.num_words): #search for the best matching word
                        distance = np.linalg.norm(des - self.words[i])
                        if distance < min_distance:
                            min_distance = distance
                            matching_word_ID = i
                    histogram[matching_word_ID] += 1
                histogram /= len(descriptors) #normalise histogram to frequencies
            histograms.append(histogram)
        return histograms

Read training files

In [2]:
import os

foods = ['Cakes', 'Pasta', 'Pizza']
path = 'FoodImages/'
training_file_names = []
training_food_labels = []

for i in range(0, len(foods)):
    sub_path = path + 'Train/' + foods[i] + '/'
    sub_file_names = [os.path.join(sub_path, f) for f in os.listdir(sub_path)]
    sub_food_labels = [i] * len(sub_file_names) #create a list of N elements, all are i
    training_file_names += sub_file_names
    training_food_labels += sub_food_labels

print(training_file_names)
print(training_food_labels)

['FoodImages/Train/Cakes/cake1.png', 'FoodImages/Train/Cakes/cake10.jpg', 'FoodImages/Train/Cakes/cake11.jpg', 'FoodImages/Train/Cakes/cake12.jpg', 'FoodImages/Train/Cakes/cake13.jpg', 'FoodImages/Train/Cakes/cake14.jpg', 'FoodImages/Train/Cakes/cake15.jpg', 'FoodImages/Train/Cakes/cake16.jpg', 'FoodImages/Train/Cakes/cake17.jpg', 'FoodImages/Train/Cakes/cake18.jpg', 'FoodImages/Train/Cakes/cake19.jpg', 'FoodImages/Train/Cakes/cake2.png', 'FoodImages/Train/Cakes/cake20.jpg', 'FoodImages/Train/Cakes/cake21.jpg', 'FoodImages/Train/Cakes/cake22.jpg', 'FoodImages/Train/Cakes/cake23.jpg', 'FoodImages/Train/Cakes/cake24.jpg', 'FoodImages/Train/Cakes/cake25.jpg', 'FoodImages/Train/Cakes/cake26.jpg', 'FoodImages/Train/Cakes/cake27.jpg', 'FoodImages/Train/Cakes/cake28.jpg', 'FoodImages/Train/Cakes/cake29.jpg', 'FoodImages/Train/Cakes/cake3.png', 'FoodImages/Train/Cakes/cake30.jpg', 'FoodImages/Train/Cakes/cake4.jpg', 'FoodImages/Train/Cakes/cake5.jpg', 'FoodImages/Train/Cakes/cake6.jpg', 'FoodI

In [4]:
num_words = 50
dictionary_name = 'food'
dictionary = Dictionary(dictionary_name, training_file_names, num_words)

In [5]:
training_word_histograms = dictionary.learn()

Save model

In [6]:
import pickle
#save dictionary
with open('food_dictionary.dic', 'wb') as f: #'wb' is for binary write
    pickle.dump(dictionary, f)

Load model

In [7]:
import pickle #you may not need to import it if this has been done
with open('food_dictionary.dic', 'rb') as f: #'rb' is for binary read
    dictionary = pickle.load(f)

KNeighborsClassifier with n=5

In [8]:
num_nearest_neighbours = 5 #number of neighbours
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours)
knn.fit(training_word_histograms, training_food_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [9]:
test_file_names = ['FoodImages/Test/Pasta/pasta35.jpg']
word_histograms = dictionary.create_word_histograms(test_file_names)
predicted_food_labels = knn.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [1]


Read test images

In [10]:
test_file_names = []
test_food_labels = []

for i in range(0, len(foods)):
    sub_path = path + 'Test/' + foods[i] + '/'
    sub_file_names = [os.path.join(sub_path, f) for f in os.listdir(sub_path)]
    sub_food_labels = [i] * len(sub_file_names) #create a list of N elements, all are i
    test_file_names += sub_file_names
    test_food_labels += sub_food_labels

print(test_file_names)
print(test_food_labels)

['FoodImages/Test/Cakes/cake31.jpg', 'FoodImages/Test/Cakes/cake32.jpg', 'FoodImages/Test/Cakes/cake33.jpg', 'FoodImages/Test/Cakes/cake34.jpg', 'FoodImages/Test/Cakes/cake35.jpg', 'FoodImages/Test/Cakes/cake36.jpg', 'FoodImages/Test/Cakes/cake37.jpg', 'FoodImages/Test/Cakes/cake38.jpg', 'FoodImages/Test/Cakes/cake39.jpg', 'FoodImages/Test/Cakes/cake40.jpg', 'FoodImages/Test/Cakes/cake41.jpg', 'FoodImages/Test/Cakes/cake42.jpg', 'FoodImages/Test/Cakes/cake43.jpg', 'FoodImages/Test/Cakes/cake44.jpg', 'FoodImages/Test/Cakes/cake45.jpg', 'FoodImages/Test/Cakes/cake46.jpg', 'FoodImages/Test/Cakes/cake47.jpg', 'FoodImages/Test/Cakes/cake48.jpg', 'FoodImages/Test/Cakes/cake49.jpg', 'FoodImages/Test/Cakes/cake50.jpg', 'FoodImages/Test/Cakes/cake51.jpg', 'FoodImages/Test/Cakes/cake52.jpg', 'FoodImages/Test/Cakes/cake53.jpg', 'FoodImages/Test/Cakes/cake54.jpg', 'FoodImages/Test/Cakes/cake55.jpg', 'FoodImages/Test/Cakes/cake56.jpg', 'FoodImages/Test/Cakes/cake57.jpg', 'FoodImages/Test/Cakes/cake

Get predicted labels for the images

In [11]:
def get_predicted_labels(classifier):
    predicted_food_labels = []
    for file_name in test_file_names:
        word_histograms = dictionary.create_word_histograms([file_name])
        predicted_food_label = classifier.predict(word_histograms)
        predicted_food_labels.append(predicted_food_label)
    return predicted_food_labels

Get classification report

In [12]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
def get_classification_report(predicted_food_labels):    
    cm = confusion_matrix(test_food_labels, predicted_food_labels)
    print('Confusion Matrix', cm)
    print(classification_report(test_food_labels, predicted_food_labels))
    print('Accuracy Score', accuracy_score(test_food_labels, predicted_food_labels))

KNeighborsClassifier with num_nearest_neighbours = 5

In [13]:
predicted_food_labels_knn = get_predicted_labels(knn)
get_classification_report(predicted_food_labels_knn)

Confusion Matrix [[19  3  8]
 [ 0 26  4]
 [ 1  5 24]]
              precision    recall  f1-score   support

           0       0.95      0.63      0.76        30
           1       0.76      0.87      0.81        30
           2       0.67      0.80      0.73        30

   micro avg       0.77      0.77      0.77        90
   macro avg       0.79      0.77      0.77        90
weighted avg       0.79      0.77      0.77        90

Accuracy Score 0.7666666666666667


KNeighborsClassifier with num_nearest_neighbours = 10

In [14]:
num_nearest_neighbours = 10 #number of neighbours
knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours)
knn.fit(training_word_histograms, training_food_labels)
predicted_food_labels_knn = get_predicted_labels(knn)
get_classification_report(predicted_food_labels_knn)

Confusion Matrix [[18  4  8]
 [ 0 27  3]
 [ 0  7 23]]
              precision    recall  f1-score   support

           0       1.00      0.60      0.75        30
           1       0.71      0.90      0.79        30
           2       0.68      0.77      0.72        30

   micro avg       0.76      0.76      0.76        90
   macro avg       0.80      0.76      0.75        90
weighted avg       0.80      0.76      0.75        90

Accuracy Score 0.7555555555555555


KNeighborsClassifier with num_nearest_neighbours = 15

In [15]:
num_nearest_neighbours = 15 #number of neighbours
knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours)
knn.fit(training_word_histograms, training_food_labels)
predicted_food_labels_knn = get_predicted_labels(knn)
get_classification_report(predicted_food_labels_knn)

Confusion Matrix [[15  5 10]
 [ 0 26  4]
 [ 0  8 22]]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        30
           1       0.67      0.87      0.75        30
           2       0.61      0.73      0.67        30

   micro avg       0.70      0.70      0.70        90
   macro avg       0.76      0.70      0.70        90
weighted avg       0.76      0.70      0.70        90

Accuracy Score 0.7


KNeighborsClassifier with num_nearest_neighbours = 20

In [16]:
num_nearest_neighbours = 20 #number of neighbours
knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours)
knn.fit(training_word_histograms, training_food_labels)
predicted_food_labels_knn = get_predicted_labels(knn)
get_classification_report(predicted_food_labels_knn)

Confusion Matrix [[13  5 12]
 [ 0 26  4]
 [ 0  8 22]]
              precision    recall  f1-score   support

           0       1.00      0.43      0.60        30
           1       0.67      0.87      0.75        30
           2       0.58      0.73      0.65        30

   micro avg       0.68      0.68      0.68        90
   macro avg       0.75      0.68      0.67        90
weighted avg       0.75      0.68      0.67        90

Accuracy Score 0.6777777777777778


KNeighborsClassifier with num_nearest_neighbours = 25

In [17]:
num_nearest_neighbours = 25 #number of neighbours
knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours)
knn.fit(training_word_histograms, training_food_labels)
predicted_food_labels_knn = get_predicted_labels(knn)
get_classification_report(predicted_food_labels_knn)

Confusion Matrix [[11  7 12]
 [ 0 26  4]
 [ 0 10 20]]
              precision    recall  f1-score   support

           0       1.00      0.37      0.54        30
           1       0.60      0.87      0.71        30
           2       0.56      0.67      0.61        30

   micro avg       0.63      0.63      0.63        90
   macro avg       0.72      0.63      0.62        90
weighted avg       0.72      0.63      0.62        90

Accuracy Score 0.6333333333333333


KNeighborsClassifier with num_nearest_neighbours = 30

In [18]:
num_nearest_neighbours = 30 #number of neighbours
knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours)
knn.fit(training_word_histograms, training_food_labels)
predicted_food_labels_knn = get_predicted_labels(knn)
get_classification_report(predicted_food_labels_knn)

Confusion Matrix [[ 9  8 13]
 [ 0 25  5]
 [ 0 10 20]]
              precision    recall  f1-score   support

           0       1.00      0.30      0.46        30
           1       0.58      0.83      0.68        30
           2       0.53      0.67      0.59        30

   micro avg       0.60      0.60      0.60        90
   macro avg       0.70      0.60      0.58        90
weighted avg       0.70      0.60      0.58        90

Accuracy Score 0.6


SVM

In [19]:
from sklearn import svm
svm_classifier = svm.SVC(C = 50, #see slide 32 in week 4 lecture slides
                            kernel = 'linear') #see slide 35 in week 4 lecture slides
svm_classifier.fit(training_word_histograms, training_food_labels)

SVC(C=50, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [20]:
test_file_name = ['FoodImages/Test/Pasta/pasta35.jpg']
word_histograms = dictionary.create_word_histograms(test_file_name)
predicted_food_labels = svm_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [1]


In [21]:
predicted_food_labels_svm = get_predicted_labels(svm_classifier)
get_classification_report(predicted_food_labels_svm)

Confusion Matrix [[26  2  2]
 [ 0 23  7]
 [ 1  5 24]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91        30
           1       0.77      0.77      0.77        30
           2       0.73      0.80      0.76        30

   micro avg       0.81      0.81      0.81        90
   macro avg       0.82      0.81      0.81        90
weighted avg       0.82      0.81      0.81        90

Accuracy Score 0.8111111111111111


SVM with C = 10

In [22]:
svm_classifier = svm.SVC(C = 10, #see slide 32 in week 4 lecture slides
                            kernel = 'linear') #see slide 35 in week 4 lecture slides
svm_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels_svm = get_predicted_labels(svm_classifier)
get_classification_report(predicted_food_labels_svm)

Confusion Matrix [[23  3  4]
 [ 0 23  7]
 [ 1  6 23]]
              precision    recall  f1-score   support

           0       0.96      0.77      0.85        30
           1       0.72      0.77      0.74        30
           2       0.68      0.77      0.72        30

   micro avg       0.77      0.77      0.77        90
   macro avg       0.78      0.77      0.77        90
weighted avg       0.78      0.77      0.77        90

Accuracy Score 0.7666666666666667


SVM with C = 20

In [23]:
svm_classifier = svm.SVC(C = 20, #see slide 32 in week 4 lecture slides
                            kernel = 'linear') #see slide 35 in week 4 lecture slides
svm_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels_svm = get_predicted_labels(svm_classifier)
get_classification_report(predicted_food_labels_svm)

Confusion Matrix [[26  2  2]
 [ 0 22  8]
 [ 1  4 25]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91        30
           1       0.79      0.73      0.76        30
           2       0.71      0.83      0.77        30

   micro avg       0.81      0.81      0.81        90
   macro avg       0.82      0.81      0.81        90
weighted avg       0.82      0.81      0.81        90

Accuracy Score 0.8111111111111111


SVM with C = 30

In [24]:
svm_classifier = svm.SVC(C = 30, #see slide 32 in week 4 lecture slides
                            kernel = 'linear') #see slide 35 in week 4 lecture slides
svm_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels_svm = get_predicted_labels(svm_classifier)
get_classification_report(predicted_food_labels_svm)

Confusion Matrix [[26  2  2]
 [ 0 22  8]
 [ 1  3 26]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91        30
           1       0.81      0.73      0.77        30
           2       0.72      0.87      0.79        30

   micro avg       0.82      0.82      0.82        90
   macro avg       0.83      0.82      0.82        90
weighted avg       0.83      0.82      0.82        90

Accuracy Score 0.8222222222222222


SVM with C = 40

In [25]:
svm_classifier = svm.SVC(C = 40, #see slide 32 in week 4 lecture slides
                            kernel = 'linear') #see slide 35 in week 4 lecture slides
svm_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels_svm = get_predicted_labels(svm_classifier)
get_classification_report(predicted_food_labels_svm)

Confusion Matrix [[26  2  2]
 [ 0 23  7]
 [ 1  5 24]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91        30
           1       0.77      0.77      0.77        30
           2       0.73      0.80      0.76        30

   micro avg       0.81      0.81      0.81        90
   macro avg       0.82      0.81      0.81        90
weighted avg       0.82      0.81      0.81        90

Accuracy Score 0.8111111111111111


AdaBoostClassifier

In [26]:
from sklearn.ensemble import AdaBoostClassifier
adb_classifier = AdaBoostClassifier(n_estimators = 150, #weak classifiers
                                        random_state = 0)
adb_classifier.fit(training_word_histograms, training_food_labels)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=150, random_state=0)

In [27]:
test_file_name = ['FoodImages/Test/Pasta/pasta35.jpg']
word_histograms = dictionary.create_word_histograms(test_file_name)
predicted_food_labels = adb_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [1]


AdaBoostClassifier with n_estimator = 150

In [28]:
predicted_food_labels_ada = get_predicted_labels(adb_classifier)
get_classification_report(predicted_food_labels_ada)

Confusion Matrix [[23  1  6]
 [ 0 16 14]
 [ 1  6 23]]
              precision    recall  f1-score   support

           0       0.96      0.77      0.85        30
           1       0.70      0.53      0.60        30
           2       0.53      0.77      0.63        30

   micro avg       0.69      0.69      0.69        90
   macro avg       0.73      0.69      0.70        90
weighted avg       0.73      0.69      0.70        90

Accuracy Score 0.6888888888888889


AdaBoostClassifier with n_estimator = 50

In [29]:
adb_classifier = AdaBoostClassifier(n_estimators = 50, #weak classifiers
                                        random_state = 0)
adb_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels_ada = get_predicted_labels(adb_classifier)
get_classification_report(predicted_food_labels_ada)

Confusion Matrix [[22  1  7]
 [ 0 18 12]
 [ 2  7 21]]
              precision    recall  f1-score   support

           0       0.92      0.73      0.81        30
           1       0.69      0.60      0.64        30
           2       0.53      0.70      0.60        30

   micro avg       0.68      0.68      0.68        90
   macro avg       0.71      0.68      0.69        90
weighted avg       0.71      0.68      0.69        90

Accuracy Score 0.6777777777777778


AdaBoostClassifier with n_estimator = 100

In [30]:
adb_classifier = AdaBoostClassifier(n_estimators = 100, #weak classifiers
                                        random_state = 0)
adb_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels_ada = get_predicted_labels(adb_classifier)
get_classification_report(predicted_food_labels_ada)

Confusion Matrix [[21  2  7]
 [ 0 14 16]
 [ 1  6 23]]
              precision    recall  f1-score   support

           0       0.95      0.70      0.81        30
           1       0.64      0.47      0.54        30
           2       0.50      0.77      0.61        30

   micro avg       0.64      0.64      0.64        90
   macro avg       0.70      0.64      0.65        90
weighted avg       0.70      0.64      0.65        90

Accuracy Score 0.6444444444444445


AdaBoostClassifier with n_estimator = 200

In [31]:
adb_classifier = AdaBoostClassifier(n_estimators = 200, #weak classifiers
                                        random_state = 0)
adb_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels_ada = get_predicted_labels(adb_classifier)
get_classification_report(predicted_food_labels_ada)

Confusion Matrix [[23  1  6]
 [ 0 16 14]
 [ 1  6 23]]
              precision    recall  f1-score   support

           0       0.96      0.77      0.85        30
           1       0.70      0.53      0.60        30
           2       0.53      0.77      0.63        30

   micro avg       0.69      0.69      0.69        90
   macro avg       0.73      0.69      0.70        90
weighted avg       0.73      0.69      0.70        90

Accuracy Score 0.6888888888888889


AdaBoostClassifier with n_estimator = 250

In [32]:
adb_classifier = AdaBoostClassifier(n_estimators = 250, #weak classifiers
                                        random_state = 0)
adb_classifier.fit(training_word_histograms, training_food_labels)
predicted_food_labels_ada = get_predicted_labels(adb_classifier)
get_classification_report(predicted_food_labels_ada)

Confusion Matrix [[24  1  5]
 [ 0 17 13]
 [ 1  8 21]]
              precision    recall  f1-score   support

           0       0.96      0.80      0.87        30
           1       0.65      0.57      0.61        30
           2       0.54      0.70      0.61        30

   micro avg       0.69      0.69      0.69        90
   macro avg       0.72      0.69      0.70        90
weighted avg       0.72      0.69      0.70        90

Accuracy Score 0.6888888888888889
